<a href="https://colab.research.google.com/github/quent1fvr/Infection_pulmonaire/blob/Quentin/1st_modele.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
import keras
import tensorflow


In [3]:
from random import randint
from numpy import array
from numpy import argmax
import keras.backend as K
from tensorflow.keras import models
from numpy import array_equal
import numpy as np
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import LSTM, Bidirectional
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras import Input
from tensorflow.keras.layers import TimeDistributed
from tensorflow.keras.layers import RepeatVector

In [ ]:
Y_normal = np.zeros(54248,dtype = np.int8)
Y_normal = np.array(Y_normal)
Y_malade = np.ones(47236,dtype = np.int8)
Y_malade = np.array(Y_malade)
Y = np.concatenate((Y_normal,Y_malade))
Y.shape

(101484,)

In [4]:
hf_normal= h5py.File("/content/drive/MyDrive/UV PROJET P6/Dataset_normal.hdf5", "r")
hf_malade= h5py.File("/content/drive/MyDrive/UV PROJET P6/Dataset_malade.hdf5", "r")

Data_normal = np.array(hf_normal["dataset_1"][:]) #dataset_name is same as hdf5 object name 
Data_malade = np.array(hf_malade["dataset_2"][:])

In [12]:
Data_normal.shape , Data_malade.shape

((55720, 48, 48), (54600, 48, 48))

In [ ]:
# 55720/70 = 796, on crée un tableau une dimension de 796 zéros en label
# 54600/70 = 780, on crée un tableau une dimension de 780 un en label
# on concatène


In [13]:
Y_normal = np.zeros(796,dtype = np.int8)
Y_malade = np.ones(780,dtype = np.int8)
Y = np.concatenate((Y_normal,Y_malade))


In [9]:
Data = np.concatenate((Data_normal,Data_malade))

In [14]:
Data = Data.reshape(1576,70,48,48) 


In [49]:
from keras.layers import Conv1D, BatchNormalization,MaxPooling1D, GlobalMaxPool1D
def build_convnet(shape=(48, 48)):
    momentum = .9
    model = keras.Sequential()
    model.add(Conv1D(64, 3, input_shape=shape,
        padding='same', activation='relu'))
    model.add(Conv1D(64, 3, padding='same', activation='relu'))
    model.add(BatchNormalization(momentum=momentum))
    
    model.add(MaxPooling1D())
    
    model.add(Conv1D(128, 3, padding='same', activation='relu'))
    model.add(Conv1D(128, 3, padding='same', activation='relu'))
    model.add(BatchNormalization(momentum=momentum))
    
    model.add(MaxPooling1D())
    
    model.add(Conv1D(256,3, padding='same', activation='relu'))
    model.add(Conv1D(256, 3, padding='same', activation='relu'))
    model.add(BatchNormalization(momentum=momentum))
    
    model.add(MaxPooling1D())
    
    model.add(Conv1D(512, 3, padding='same', activation='relu'))
    model.add(Conv1D(512, 3, padding='same', activation='relu'))
    model.add(BatchNormalization(momentum=momentum))
    
    # flatten...
    model.add(GlobalMaxPool1D())
    return model

In [50]:
from keras.layers import TimeDistributed, GRU, Dense, Dropout
def action_model(shape=(70,48, 48), nbout=1):
    # Create our convnet with (112, 112, 3) input shape
    convnet = build_convnet(shape[1:])
    
    # then create our final model
    model = keras.Sequential()
    # add the convnet with (70,48,48,3) shape
    model.add(TimeDistributed(convnet, input_shape=(70,48, 48)))
    # here, you can also use GRU or LSTM
    model.add(GRU(64))
    # and finally, we make a decision network
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(.5))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(.5))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(.5))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(nbout, activation='sigmoid'))
    return model

In [55]:
# some global params
SIZE = (48, 48)
NBFRAME = 70
BS = 35

In [52]:
INSHAPE=(NBFRAME,) + SIZE  # (70, 48, 48)
print(INSHAPE)
model = action_model(INSHAPE, nbout=1)
optimizer = tensorflow.keras.optimizers.Adam(0.001)
model.compile(
    optimizer,
    'binary_crossentropy',
    metrics=['acc']
)

(70, 48, 48)


In [56]:
model.fit(Data,Y,epochs=20, validation_split=0.2, batch_size=BS)

Epoch 1/20
36/36 [==============================] - 12s 174ms/step - loss: 0.1450 - acc: 0.9437 - val_loss: 0.2375 - val_acc: 0.8892
Epoch 2/20
36/36 [==============================] - 6s 162ms/step - loss: 0.1761 - acc: 0.9357 - val_loss: 0.2925 - val_acc: 0.8987
Epoch 3/20
36/36 [==============================] - 6s 163ms/step - loss: 0.2185 - acc: 0.9159 - val_loss: 0.2743 - val_acc: 0.8797
Epoch 4/20
36/36 [==============================] - 6s 164ms/step - loss: 0.1626 - acc: 0.9405 - val_loss: 0.1214 - val_acc: 0.9494
Epoch 5/20
36/36 [==============================] - 6s 165ms/step - loss: 0.1438 - acc: 0.9460 - val_loss: 0.2224 - val_acc: 0.9241
Epoch 6/20
36/36 [==============================] - 6s 164ms/step - loss: 0.1354 - acc: 0.9500 - val_loss: 0.2072 - val_acc: 0.9241
Epoch 7/20
36/36 [==============================] - 6s 163ms/step - loss: 0.1715 - acc: 0.9365 - val_loss: 0.2300 - val_acc: 0.8987
Epoch 8/20
36/36 [==============================] - 6s 162ms/step - loss: 0